In [200]:
import pandas as pd
import plotly.graph_objects as go
import requests
import matplotlib.pyplot as plt
import json
import matplotlib.animation as animation
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

I scrapped test batting records of Pujara, Rohit, Kohli and Rahane from their respective cricinfo pages. For reference page of rohit sharma is given here https://stats.espncricinfo.com/ci/engine/player/34102.html?class=1;template=results;type=batting;view=innings

In [201]:
pujara = pd.read_csv('pujaratest.csv')

In [202]:
pujara.head()

,Runs,Mins,BF,4s,6s,SR,Pos,Dismissal,Inns,Unnamed: 9,Opposition,Ground,Start Date,Unnamed: 13
0,4,3,3,1,0,133.33,5,lbw,2,NaN,v Australia,Bengaluru,9 Oct 2010,Test # 1973
1,72,129,89,7,0,80.89,3,bowled,4,NaN,v Australia,Bengaluru,9 Oct 2010,Test # 1973
2,19,54,45,3,0,42.22,6,caught,1,NaN,v South Africa,Durban,26 Dec 2010,Test # 1987
3,10,81,56,1,0,17.85,6,bowled,3,NaN,v South Africa,Durban,26 Dec 2010,Test # 1987
4,2,17,14,0,0,14.28,6,lbw,2,NaN,v South Africa,Cape Town,2 Jan 2011,Test # 1988


In [203]:
def processtest(pujara,name):
    pujara['notout']= pujara.Runs.str.contains('DNB')|pujara.Runs.str.contains('\*')
    pujara.Runs = np.where(pujara.Runs.str.contains('DNB'),0,pujara.Runs)
    pujara.Runs = pujara.Runs.str.replace('\*','')
    pujara.Runs = pujara.Runs.fillna(0)
    pujara.Runs = pujara.Runs.astype(int)
    pujara['totalruns'] =pujara.Runs.cumsum()
    pujara['out'] =~pujara.notout
    pujara['completedinn']=pujara.out.cumsum()
    pujara['avg'] =pujara.totalruns/pujara.completedinn
    pujara['Date']= pd.to_datetime(pujara['Start Date'])
    pujara = pujara.drop_duplicates(subset='Date', keep='last')
    pujara['Date2']=pujara.Date.dt.strftime('%d/%m/%y')
    pujara['name'] = name
    return pujara

In [204]:
pujara = processtest(pujara,'Pujara')

/home/rohit/Desktop/virtual_environments/sebi/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/rohit/Desktop/virtual_environments/sebi/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [205]:
kohli = pd.read_csv('kohlitest.csv')

In [206]:
kohli = processtest(kohli,'Kohli')

/home/rohit/Desktop/virtual_environments/sebi/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/rohit/Desktop/virtual_environments/sebi/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [207]:
rahane = pd.read_csv('rahanetest.csv')

In [208]:
rahane = processtest(rahane,'Rahane')

/home/rohit/Desktop/virtual_environments/sebi/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/rohit/Desktop/virtual_environments/sebi/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [209]:
rohit = pd.read_csv('rohittest.csv')

In [210]:
rohit = processtest(rohit,'Rohit')

/home/rohit/Desktop/virtual_environments/sebi/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/rohit/Desktop/virtual_environments/sebi/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [211]:
def createfinaldf(listdf):
    datesindex = np.unique(np.hstack([x.Date.values for x in listdf]))
    indexdf = pd.DataFrame({'Date':datesindex})
    listdf2 = []
    for x in listdf:
        merged = pd.merge(x,indexdf,how='right', on='Date' )
        merged = merged.sort_values('Date')
        merged.name = merged.name.fillna(method='bfill')
        merged = merged.fillna(method='ffill')
        merged = merged.fillna(0)
        listdf2.append(merged)
    mergedf = pd.concat(listdf2)
    mergedf['Date2']=mergedf.Date.dt.strftime('%d/%m/%y')
    mergedf['legend']= mergedf.name
    return mergedf


In [212]:
finaldf =createfinaldf([pujara,rohit,kohli, rahane])

finaldf

,Runs,Mins,BF,4s,6s,SR,Pos,Dismissal,Inns,Unnamed: 9,...,Unnamed: 13,notout,totalruns,out,completedinn,avg,Date,Date2,name,legend
0,72.0,129,89,7,0,80.89,3,bowled,4,0.0,...,Test # 1973,False,76.0,True,2.0,38.000000,2010-10-09,09/10/10,Pujara,Pujara
1,10.0,81,56,1,0,17.85,6,bowled,3,0.0,...,Test # 1987,False,105.0,True,4.0,26.250000,2010-12-26,26/12/10,Pujara,Pujara
2,0.0,-,-,-,-,-,-,-,4,0.0,...,Test # 1988,True,107.0,False,5.0,21.400000,2011-01-02,02/01/11,Pujara,Pujara
103,0.0,-,-,-,-,-,-,-,4,0.0,...,Test # 1988,True,107.0,False,5.0,21.400000,2011-06-20,20/06/11,Pujara,Pujara
104,0.0,-,-,-,-,-,-,-,4,0.0,...,Test # 1988,True,107.0,False,5.0,21.400000,2011-06-28,28/06/11,Pujara,Pujara
105,0.0,-,-,-,-,-,-,-,4,0.0,...,Test # 1988,True,107.0,False,5.0,21.400000,2011-07-06,06/07/11,Pujara,Pujara
106,0.0,-,-,-,-,-,-,-,4,0.0,...,Test # 1988,True,107.0,False,5.0,21.400000,2011-11-22,22/11/11,Pujara,Pujara
107,0.0,-,-,-,-,-,-,-,4,0.0,...,Test # 1988,True,107.0,False,5.0,21.400000,2011-12-26,26/12/11,Pujara,Pujara
108,0.0,-,-,-,-,-,-,-,4,0.0,...,Test # 1988,True,107.0,False,5.0,21.400000,2012-01-03,03/01/12,Pujara,Pujara
109,0.0,-,-,-,-,-,-,-,4,0.0,...,Test # 1988,True,107.0,False,5.0,21.400000,2012-01-13,13/01/12,Pujara,Pujara


In [213]:
fig = px.scatter(finaldf,x='totalruns', y = 'avg', animation_frame='Date2', text='name', color= 'name',animation_group='name',range_x=[0,10000], range_y=[0,70])
fig.update_layout(showlegend=True)
fig.update_traces(marker=dict(size=30))
fig.write_html("chart.html")
